<a href="https://colab.research.google.com/github/jmanning21/31005_MachineLearning_Python/blob/master/Machine_Learning_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2: Practical Machine Learning Project
####Affnan Amir (13528841) and Julia Manning (12875795)

###Introduction
The algorithm chosen for this project is the ID3 Decision Tree building algorithm. In using this alrogithm, the data  

* Input = 
* Output = 

###Exploration

Challenges:
 - over-fitting the data

###Methodology

In [0]:
# Importing the required packages
import numpy as np
import pandas as pd

In [0]:
# Importing the dataset from a url
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

# Read the csv and convert to a dataframe
d = pd.read_csv(url)
df = pd.DataFrame(data=d)

# See the first ten rows of the dataset
df.head(10)


In [0]:
# Preprocessing some of the attributes: replace null values with median values, and changing variable type to reflect the real-world scenario
# Replacing null values in age
 

###Evaluation

###Conclusion

###Ethical Justification

- potential misuses of the technique
- utilitarian view?

###Video Pitch
*Attached to the final document*